In [13]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt
import sys

In [14]:
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR

In [15]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.permutations import RandomPermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline
from torch.utils.tensorboard import SummaryWriter

In [16]:
import subprocess
import time
import os
from copy import deepcopy
import math as m
import gc

## Tensorboard writer for loss logging

In [ ]:
writer = SummaryWriter()

## GPU/CPU selection

In [ ]:
device = torch.device("cuda:1")
#device = torch.device("cpu")

## Hyperparameters

In [ ]:
n_RQS_knots = 10   # Number of knots in RQS transform
n_made_layers = 1  # Number of hidden layers in every made network
n_made_units = 100 # Number of units in every layer of the made network
n_flow_layers = 6  # Number of layers in the flow

batch_size = 1024
n_epochs = 800
adam_lr = 0.001     # Learning rate for the ADAM optimizer (default: 0.001)

n_train = int(1e6)  # Number of training events
n_test = int(1e5)   # Number of testing events
n_sample = int(1e5) # Number of samples for ess evaluation

## Load the training data

In [ ]:
samples = np.genfromtxt("data/unweighted_samples.csv", delimiter=',')
if (n_train + n_test > samples.shape[0]):
    raise Exception("Not enough training data")

## Split to a train and test set

In [ ]:
train_samples = torch.tensor(samples[:n_train], dtype=torch.float32, device=device)
test_samples = torch.tensor(samples[n_train:n_train+n_test], dtype=torch.float32, device=device)

del samples
gc.collect()

67

## Set up the flow

In [ ]:
event_dim = train_samples.shape[1]
base_dist = BoxUniform(torch.zeros(event_dim), torch.ones(event_dim))

transforms = []
for _ in range(n_flow_layers):
    transforms.append(RandomPermutation(features=event_dim))
    transforms.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=event_dim, 
        hidden_features=n_made_units,
        num_bins=n_RQS_knots,
        num_blocks=n_made_layers-1,
        tails="constrained",
        use_residual_blocks=False
    ))
transform = CompositeTransform(transforms)

flow = Flow(transform, base_dist).to(device)
optimizer = optim.Adam(flow.parameters(), lr=adam_lr)

scheduler = MultiStepLR(optimizer, milestones=[350, 425, 500, 575, 650, 725, 800], gamma=0.5)

## Training

In [ ]:
data_size = train_samples.shape[0]
n_batches = m.ceil(data_size/batch_size)

data_size_validation = test_samples.shape[0]
n_batches_validate = m.ceil(data_size_validation/batch_size)

best_validation_loss = np.inf
best_ess = 0

for epoch in range(n_epochs):
    
    permutation = torch.randperm(data_size, device=device)    

    # Loop over batches
    cum_loss = 0
    for batch in range(n_batches):
        # Set up the batch
        batch_begin = batch*batch_size
        batch_end   = min( (batch+1)*batch_size, data_size-1 )
        indices = permutation[batch_begin:batch_end]
        samples_batch = train_samples[indices]
        
        # Take a step
        optimizer.zero_grad()
        loss = -(flow.log_prob(inputs=samples_batch)).mean()
        loss.backward()
        optimizer.step()

        # Compute cumulative loss
        cum_loss = (cum_loss*batch + loss.item())/(batch+1)

        if batch%25 == 0:
            print("epoch = ", epoch, "batch = ", batch, "/", n_batches, "loss = ", cum_loss)
    
    writer.add_scalar("Loss_train", cum_loss, epoch)
    scheduler.step()


    # ---------- Compute validation loss -----------
    validation_loss = 0
    for batch in range(n_batches_validate):
        batch_begin = batch*batch_size
        batch_end = min( (batch+1)*batch_size, data_size_validation-1 )
        samples_batch = test_samples[batch_begin:batch_end]
    
        with torch.no_grad():
            validation_loss = (validation_loss*batch - (flow.log_prob(samples_batch)).mean())/(batch+1)

    print("Validation loss = ", validation_loss.item())
    writer.add_scalar("Loss_test", validation_loss.item(), epoch)

    if validation_loss < best_validation_loss:
        torch.save(flow, "flow_model_unweighted_best_validation.pt")
        best_validation_loss = validation_loss

    
    # ---------- Compute effective sample size ----------
    # generate samples and evaluate llhs
    with torch.no_grad():
        samples = flow.sample(n_sample)
        llhs = flow.log_prob(samples)

    # Store files
    np.savetxt("/tmp/samples_file.csv", samples.cpu().numpy(), delimiter=',')
    np.savetxt("/tmp/llhs_file.csv", np.exp(llhs.cpu().numpy()), delimiter=',')

    # Run the evaluator
    cmd = os.path.abspath(os.getcwd())+'/ME_VEGAS/compute_metrics_from_likelihoods /tmp/samples_file.csv /tmp/llhs_file.csv'
    b = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE).stdout
    lines = b.decode('ascii').split("\n")

    ess = float(lines[2].split(' ')[-1])
    
    print("Effective sample size = ", ess)
    writer.add_scalar("Effective_sample_size", ess, epoch)

    if ess > best_ess:
        torch.save(flow, "flow_model_unweighted_best_ess.pt")
        best_ess = ess
        
torch.save(flow, "flow_model_unweighted_final.pt")

epoch =  0 batch =  0 / 977 loss =  0.36800092458724976
epoch =  0 batch =  25 / 977 loss =  -4.81019204797653
epoch =  0 batch =  50 / 977 loss =  -7.429467955932896
epoch =  0 batch =  75 / 977 loss =  -9.656738750048373
epoch =  0 batch =  100 / 977 loss =  -11.553332369958998
epoch =  0 batch =  125 / 977 loss =  -13.018490877298134
epoch =  0 batch =  150 / 977 loss =  -14.256637354461562
epoch =  0 batch =  175 / 977 loss =  -15.289999546567826
epoch =  0 batch =  200 / 977 loss =  -16.13614581169477
epoch =  0 batch =  225 / 977 loss =  -16.82193415717478
epoch =  0 batch =  250 / 977 loss =  -17.384208729958168
epoch =  0 batch =  275 / 977 loss =  -17.858309729753646
epoch =  0 batch =  300 / 977 loss =  -18.262923184769907
epoch =  0 batch =  325 / 977 loss =  -18.609695450248548
epoch =  0 batch =  350 / 977 loss =  -18.9011194314967
epoch =  0 batch =  375 / 977 loss =  -19.167159768772585
epoch =  0 batch =  400 / 977 loss =  -19.396337840585357
epoch =  0 batch =  425 / 9

epoch =  3 batch =  475 / 977 loss =  -23.405365010269556
epoch =  3 batch =  500 / 977 loss =  -23.407192241645866
epoch =  3 batch =  525 / 977 loss =  -23.407611176088288
epoch =  3 batch =  550 / 977 loss =  -23.409256749923355
epoch =  3 batch =  575 / 977 loss =  -23.409335338407104
epoch =  3 batch =  600 / 977 loss =  -23.409225086205815
epoch =  3 batch =  625 / 977 loss =  -23.409449327487163
epoch =  3 batch =  650 / 977 loss =  -23.4092820251043
epoch =  3 batch =  675 / 977 loss =  -23.407976731746164
epoch =  3 batch =  700 / 977 loss =  -23.409137366671715
epoch =  3 batch =  725 / 977 loss =  -23.40938885743954
epoch =  3 batch =  750 / 977 loss =  -23.410999828902455
epoch =  3 batch =  775 / 977 loss =  -23.411625835084433
epoch =  3 batch =  800 / 977 loss =  -23.412218939201374
epoch =  3 batch =  825 / 977 loss =  -23.409809632105063
epoch =  3 batch =  850 / 977 loss =  -23.411027950909958
epoch =  3 batch =  875 / 977 loss =  -23.411486893484046
epoch =  3 batch 

epoch =  6 batch =  950 / 977 loss =  -23.504544654228464
epoch =  6 batch =  975 / 977 loss =  -23.503706087831596
Validation loss =  -23.51233673095703
Effective sample size =  0.385979
epoch =  7 batch =  0 / 977 loss =  -23.685314178466797
epoch =  7 batch =  25 / 977 loss =  -23.513135029719425
epoch =  7 batch =  50 / 977 loss =  -23.506691053801887
epoch =  7 batch =  75 / 977 loss =  -23.50863308655588
epoch =  7 batch =  100 / 977 loss =  -23.51628041031337
epoch =  7 batch =  125 / 977 loss =  -23.51639723399329
epoch =  7 batch =  150 / 977 loss =  -23.51824761700157
epoch =  7 batch =  175 / 977 loss =  -23.517819957299672
epoch =  7 batch =  200 / 977 loss =  -23.51580271554824
epoch =  7 batch =  225 / 977 loss =  -23.512810226035334
epoch =  7 batch =  250 / 977 loss =  -23.508142805669415
epoch =  7 batch =  275 / 977 loss =  -23.507923540861714
epoch =  7 batch =  300 / 977 loss =  -23.509312195635324
epoch =  7 batch =  325 / 977 loss =  -23.513172974615753
epoch =  7

epoch =  10 batch =  400 / 977 loss =  -23.550535073601402
epoch =  10 batch =  425 / 977 loss =  -23.551118613408768
epoch =  10 batch =  450 / 977 loss =  -23.550273463889926
epoch =  10 batch =  475 / 977 loss =  -23.551137996320968
epoch =  10 batch =  500 / 977 loss =  -23.55113951174799
epoch =  10 batch =  525 / 977 loss =  -23.551414156141394
epoch =  10 batch =  550 / 977 loss =  -23.55144124455115
epoch =  10 batch =  575 / 977 loss =  -23.551177521546695
epoch =  10 batch =  600 / 977 loss =  -23.55071445471437
epoch =  10 batch =  625 / 977 loss =  -23.550067575594877
epoch =  10 batch =  650 / 977 loss =  -23.5494300238731
epoch =  10 batch =  675 / 977 loss =  -23.549393989630723
epoch =  10 batch =  700 / 977 loss =  -23.549039522353347
epoch =  10 batch =  725 / 977 loss =  -23.548361660035212
epoch =  10 batch =  750 / 977 loss =  -23.54870398987469
epoch =  10 batch =  775 / 977 loss =  -23.548067916299903
epoch =  10 batch =  800 / 977 loss =  -23.547802126214172
epo